In [2]:
import cohere
import json
import io
import warnings

import pandas as pd
from IPython.display import display
from PIL import Image

from stability_sdk import client
import stability_sdk.interfaces.gooseai.generation.generation_pb2 as generation

# APIs

In [3]:
with open('../cohere_api_key.txt', 'r') as f:
    cohere_api_key = f.read()
co = cohere.Client(cohere_api_key)

with open('../stability_api_key.txt', 'r') as f:
    stability_api_key = f.read()
stability_api = client.StabilityInference(
    key=stability_api_key, 
    verbose=True,
)

del cohere_api_key
del stability_api_key

# EDA

In [4]:
with open('../stories/hansel_and_gretel.json') as f:
    data = json.load(f)
    text = data['text']
    print(type(text), len(text))

FileNotFoundError: [Errno 2] No such file or directory: '../stories/hansel_and_gretel.json'

In [ ]:
with open('../stories/red_riding_hood.json') as f:
    data = json.load(f)
    text = data['text']
    print(type(text), len(text))

<class 'str'> 3645


In [ ]:
with open('../stories/three_little_pigs.json', errors='ignore') as f:
    data = json.load(f)
    text = data['text']
    print(type(text), len(text))

<class 'str'> 3403


# Generating stories

In [5]:
def generate(prompt, model="xlarge", 
             num_generations=5, temperature=0.7, 
             max_tokens=2000, stop_sequences=['<end>']):
             
  prediction = co.generate(
    model=model,
    prompt=prompt,
    return_likelihoods = 'GENERATION',
    stop_sequences=stop_sequences,
    max_tokens=max_tokens,
    temperature=temperature,
    num_generations=num_generations)
  
  # Get list of generations
  gens = []
  likelihoods = []
  for gen in prediction.generations:
      gens.append(gen.text)
      
      sum_likelihood = 0
      for t in gen.token_likelihoods:
          sum_likelihood += t.likelihood
      # Get sum of likelihoods
      likelihoods.append(sum_likelihood)

  pd.options.display.max_colwidth = 200
  # Create a dataframe for the generated sentences and their likelihood scores
  df = pd.DataFrame({'generation':gens, 'likelihood': likelihoods})
  # Drop duplicates
  df = df.drop_duplicates(subset=['generation'])
  # Sort by highest sum likelihood
  df = df.sort_values('likelihood', ascending=False, ignore_index=True)
  
  return df

In [6]:
def generate_image(image_prompt):
  # the object returned is a python generator
  answers = stability_api.generate(
      prompt=image_prompt
  )

  # iterating over the generator produces the api response
  for resp in answers:
      for artifact in resp.artifacts:
          if artifact.finish_reason == generation.FILTER:
              warnings.warn(
                  "Your request activated the API's safety filters and could not be processed."
                  "Please modify the prompt and try again.")
          if artifact.type == generation.ARTIFACT_IMAGE:
              img = Image.open(io.BytesIO(artifact.binary))
              display(img)


In [7]:
result = generate(
    "short fairy tale about a girl",
    max_tokens=1000
)

In [10]:
print(result.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   generation  5 non-null      object 
 1   likelihood  5 non-null      float64
dtypes: float64(1), object(1)
memory usage: 208.0+ bytes
None


In [12]:
print(result['generation'][0])

 who's trying to put on a dress. But
as the story progresses, it's revealed that she's putting on a
dress of flowers, and that she's a princess who is going to marry a
frog.

The story has a pretty simple
plot, but it's also a pretty good one. It's one of those stories that
I've read over and over again, and it never gets old. I also like the
story because it's very funny. I can't help but laugh when the
princess gets stuck in her dress.

This story is very similar to
the original, but it's also a bit different. In this story, the girl
is not a princess, but she is a fairy. She is trying to put on a
dress, but she is wearing a dress of flowers. She is also not a
princess, but she is a princess.

This is a story that is very similar to the original, but it is a bit different. In this story, the girl is not a princess, but she is a fairy. She is trying to put on a dress, but she is wearing a dress of flowers. She is also not a princess, but she is a princess.

I like this story because i